# 패키지 설치
pip 명령어로 의존성 있는 패키지를 설치합니다.



In [15]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [16]:
%cd /content/drive/MyDrive/recipekogpt2

/content/drive/.shortcut-targets-by-id/1jOKmNyG5BEUAKKmGZ__qNyXG3qcjgYHB/recipekogpt2


In [4]:
!pip install -r requirements.txt

     |████████████████████████████████| 808kB 6.9MB/s 
     |████████████████████████████████| 2.3MB 29.9MB/s 
     |████████████████████████████████| 61kB 8.1MB/s 
--- Logging error ---
Traceback (most recent call last):
  File "/usr/lib/python3.7/logging/__init__.py", line 1025, in emit
    msg = self.format(record)
  File "/usr/local/lib/python3.7/dist-packages/pip/_internal/utils/logging.py", line 225, in format
    msg = logging.StreamHandler.format(self, record)
  File "/usr/lib/python3.7/logging/__init__.py", line 869, in format
    return fmt.format(record)
  File "/usr/local/lib/python3.7/dist-packages/pip/_internal/utils/logging.py", line 151, in format
    formatted = super(IndentingFormatter, self).format(record)
  File "/usr/lib/python3.7/logging/__init__.py", line 608, in format
    record.message = record.getMessage()
  File "/usr/lib/python3.7/logging/__init__.py", line 369, in getMessage
    msg = msg % self.args
  File "/usr/local/lib/python3.7/dist-packages/pip/_inte

# 각종 설정
모델 하이퍼파라메터(hyperparameter)와 저장 위치 등 설정 정보를 선언합니다.
downstream_model_dir에 정보를 입력하면 파인튜닝을 마친 모델로 인퍼런스를 합니다.

In [17]:
from ratsnlp.nlpbook.generation import GenerationDeployArguments
args = GenerationDeployArguments(
    pretrained_model_name="skt/kogpt2-base-v2",
    downstream_model_dir="./model_checkpoints",
)

downstream_model_checkpoint_fpath: ./model_checkpoints/epoch=1-val_loss=2.28.ckpt


# 모델 로딩
프리트레인한(혹은 파인튜닝을 마친) GPT2 모델과 토크나이저를 읽어 들입니다.

In [18]:
import torch
from transformers import GPT2Config, GPT2LMHeadModel
if args.downstream_model_checkpoint_fpath is None:
    model = GPT2LMHeadModel.from_pretrained(
        args.pretrained_model_name,
    )
else:
    pretrained_model_config = GPT2Config.from_pretrained(
        args.pretrained_model_name,
    )
    model = GPT2LMHeadModel(pretrained_model_config)
    fine_tuned_model_ckpt = torch.load(
        args.downstream_model_checkpoint_fpath,
        map_location=torch.device("cpu"),
    )
    model.load_state_dict({k.replace("model.", ""): v for k, v in fine_tuned_model_ckpt['state_dict'].items()})
model.eval()

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(51200, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )


In [19]:
tokens_list = ['<unused0>','<unused1>','<unused2>','<unused3>','<unused4>','<unused5>']
from transformers import PreTrainedTokenizerFast
tokenizer = PreTrainedTokenizerFast.from_pretrained("skt/kogpt2-base-v2",
  bos_token='</s>', eos_token='</s>', unk_token='<unk>',
  pad_token='<pad>', mask_token='<mask>', additional_special_tokens = tokens_list)

In [20]:
input_ids = tokenizer.encode('<unused0>간장 계란밥<unused1><unused2>간장$참기름$쌀$계란$<unused3><unused4>', return_tensors='pt')
input_ids

tensor([[    9, 49375,  9142,  7374,  7605,    10,    11, 49375,   379,  8340,
         19824,   379,  7902,   379,  6886,  7374,   379,    12,    13]])

In [21]:
import torch
with torch.no_grad():
    generated_ids = model.generate(
        input_ids,
        do_sample=False,
        min_length=50,
        max_length=100,
    )
print(tokenizer.decode([el.item() for el in generated_ids[0]]))

<unused0> 간장 계란밥<unused1><unused2> 간장$참기름$쌀$계란$<unused3><unused4> 간장 참기름을 섞어준다. 밥에 참기름 계란 밥을 넣고 잘 섞어준다. 밥에 참기름 계란 밥을 넣고 잘 섞어준다.<unused5><unused2> 밥$공기$계란$<unused3><unused4><unused5></s>


In [22]:
with torch.no_grad():
    generated_ids = model.generate(
        input_ids,
        do_sample=False,
        min_length=10,
        max_length=120,
        num_beams=3,
    )
print(tokenizer.decode([el.item() for el in generated_ids[0]]))

<unused0> 간장 계란밥<unused1><unused2> 간장$참기름$쌀$계란$<unused3><unused4> 간장 참기름에 계란을 풀어준다. 밥에 참기름과 쌀 계란 물을 넣고 섞어준다. 밥에 참기름과 쌀 계란 물을 넣고 섞어준다. 밥에 계란 물을 입혀 완성한다.<unused5></s>


In [23]:
with torch.no_grad():
    generated_ids = model.generate(
        input_ids,
        do_sample=False,
        min_length=10,
        max_length=120,
        repetition_penalty=1.5,
    )
print(tokenizer.decode([el.item() for el in generated_ids[0]]))

<unused0> 간장 계란밥<unused1><unused2> 간장$참기름$쌀$계란$<unused3><unused4> 밥솥에 참기름을 두르고 밥을 짓는다. 쌀 1컵을 씻어 불린 후 체에 내린 뒤 참기름과 섞는 다 그릇에 담아낸다<unused5></s>


In [27]:
with torch.no_grad():
    generated_ids = model.generate(
        input_ids,
        do_sample=True,
        min_length=50,
        max_length=150,
        repetition_penalty=1.5,
        no_repeat_ngram_size=3,
        temperature=0.9,
        top_k=50,
        top_p=0.92,
    )
print(tokenizer.decode([el.item() for el in generated_ids[0]]))

<unused0> 간장 계란밥<unused1><unused2> 간장$참기름$쌀$계란$<unused3><unused4> 이번에는 초간단 레시피를 보여줄 이색 요리를 소개해볼게요 재료 간장 1큰 술 참기름 2 큰 름 쌀 14컵 밥 1인분 양념장 양파 12 청양고추 13 마늘 15 작은 술 쪽파나베 멸치볶음 20작은 술 설탕 약간 양파는 다지고 당근 호박고추는 어슷하게 썰어주세요. 팬에 참기름을 두르고 다진 파를 넣고 볶아주 다가 쌀을 넣어준 후 국간장을 풀어주고 밥을 추가해주면 완성 달군 프라이팬에 기름을 살짝 두른 뒤 편으로 썬 감자를 올려서 스크램블을 만들어줍니다.<unused5></s>


# 인퍼런스 함수 선언
인퍼런스 함수를 선언합니다.

In [13]:
def inference_fn(
        prompt,
        min_length=10,
        max_length=100,
        top_p=1.0,
        top_k=50,
        repetition_penalty=1.0,
        no_repeat_ngram_size=0,
        temperature=1.0,
):
    try:
        input_ids = tokenizer.encode(prompt, return_tensors="pt")
        with torch.no_grad():
            generated_ids = model.generate(
                input_ids,
                do_sample=True,
                top_p=float(top_p),
                top_k=int(top_k),
                min_length=int(min_length),
                max_length=int(max_length),
                repetition_penalty=float(repetition_penalty),
                no_repeat_ngram_size=int(no_repeat_ngram_size),
                temperature=float(temperature),
           )
        generated_sentence = tokenizer.decode([el.item() for el in generated_ids[0]])
    except:
        generated_sentence = """처리 중 오류가 발생했습니다. <br>
            변수의 입력 범위를 확인하세요. <br><br> 
            min_length: 1 이상의 정수 <br>
            max_length: 1 이상의 정수 <br>
            top-p: 0 이상 1 이하의 실수 <br>
            top-k: 1 이상의 정수 <br>
            repetition_penalty: 1 이상의 실수 <br>
            no_repeat_ngram_size: 1 이상의 정수 <br>
            temperature: 0 이상의 실수
            """
    return {
        'result': generated_sentence,
    }

# 웹서비스 개시
아래처럼 실행해 인퍼런스 함수를 웹서비스로 만듭니다.

In [14]:
from ratsnlp.nlpbook.generation import get_web_service_app
app = get_web_service_app(inference_fn)
app.run()

 * Serving Flask app "ratsnlp.nlpbook.generation.deploy" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://aadbc087938c.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [16/Jun/2021 10:49:32] "GET / HTTP/1.1" 200 -
